In [6]:
import os
import json
from tqdm import tqdm
import torch
import numpy as np
from openai import OpenAI
import backoff

In [3]:
port = 8002
model = 'Qwen/QwQ-32B'

OPENAI_REQUEST_TIMEOUT = 60*60*24 
client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="EMPTY", timeout=OPENAI_REQUEST_TIMEOUT)
print(client.models.list())

# @backoff.on_exception(backoff.constant, Exception, interval=5)
def run_chat_completion_with_backoff(client, **kwargs):
    return client.chat.completions.create(**kwargs)


@backoff.on_exception(backoff.constant, Exception, interval=5)
def run_generate_with_backoff(client, **kwargs):
    return client.completions.create(**kwargs)

SyncPage[Model](data=[Model(id='Qwen/QwQ-32B', created=1751909088, object='model', owned_by='vllm', root='Qwen/QwQ-32B', parent=None, max_model_len=40960, permission=[{'id': 'modelperm-b03021f5449e45e2baaa9140edbbc1d3', 'object': 'model_permission', 'created': 1751909088, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}])], object='list')


In [4]:
# gpqa
logs_direct_pred = json.load(open('/fsx-comem/diwu0162/Search-o1/outputs/runs.analysis/gpqa.qwq.search_o1.10.10/diamond.7.2,20:28.json'))
logs_search_o1 = json.load(open('/fsx-comem/diwu0162/WebThinker/outputs/runs.analysis/gpqa.qwq.search_o1.10.10/diamond.7.7,2:19.filtered.json'))
logs_webthinker = json.load(open('/fsx-comem/diwu0162/WebThinker/outputs/gpqa.qwq.webthinker/diamond.7.7,2:26.92.filtered.json'))
assert len(logs_direct_pred) == len(logs_search_o1) == len(logs_webthinker)

In [5]:
# logs_direct_pred[0]['Question']
# logs_search_o1[1]['Question']
# logs_webthinker[5]['Question']
logs_direct_pred[0]['Question']

'<|im_start|>user\nYou are a reasoning assistant with the ability to perform web searches to help you answer the user\'s question accurately. You have special tools:\n\n- To perform a search: write <|begin_search_query|> your query here <|end_search_query|>.\nThen, the system will search and analyze relevant web pages, then provide you with helpful information in the format <|begin_search_result|> ...search results... <|end_search_result|>.\n\nYou can repeat the search process multiple times if necessary. The maximum number of search attempts is limited to 10.\n\nOnce you have all the information you need, continue your reasoning.\n\nExample:\nQuestion: "What is the energy range of pp III neutrinos?"\nAssistant thinking steps:\n- I might need to look up details about pp III neutrinos.\n\nAssistant:\n<|begin_search_query|>pp III neutrino energy spectrum<|end_search_query|>\n\n(System returns processed information from relevant web pages)\n\nAssistant continues reasoning with the new inf